# The Fast Fourier Transform

This notebook explores the ideas behind the Fast Fourier Transform or FFT as a method of _quickly_ calculating the product of two polynomials.

## Setup

We first do some imports of common libraries. While libraries like NumPy already contain a way more efficient [FFT](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html) implementation, the purpose of this notebook is to write our own (very limit) version.

In [ ]:
import numpy as np

## Little Helpers

Small helper functions to check if $n$ is a power of $2$.

In [ ]:
def ispow2(n: int):
    return (n & (n - 1) == 0) and n != 0

## Implementing the FFT

Here we finally implement our very simple variant of the FFT. We only handle inputs of length $2^k, k \in \mathbb{N}_{0}$. Other implementations of FFT exist, in which the input can be of any length (such as NumPy's [`numpy.fft.fff()`](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html)), but those are a little more complicated.

In [ ]:
def fft(polynomial: np.typing.ArrayLike):
    polynomial = np.asarray(polynomial)
    n = len(polynomial)
    # assure order of polynomial is power of 2
    assert ispow2(n)

    # ensure recurson stops!
    if n == 1:
        return polynomial

    unity_root = np.exp(2j * np.pi / n)

    # select every other element (0, 2, 4, ...)
    polynomial_even = polynomial[::2]
    # select every other element ... starting from 1 (1, 3, 5, ...)
    polynomial_odd = polynomial[1::2]

    y_even, y_odd = fft(polynomial_even), fft(polynomial_odd)

    y = np.empty(n, dtype=complex)
    for k in range(n // 2):
        y[k] = y_even[k] + unity_root ** -k * y_odd[k]
        y[k + n // 2] = y_even[k] - unity_root ** -k * y_odd[k]

    return y

## Testing

Play through some values and check against a _trusted_ FFT implementation.

In [ ]:
polynomial = range(2 ** 3)

# Let's test our FFT
a = fft(polynomial)

# For comparison reasons, let's also try NumPy's FFT
b = np.fft.fft(polynomial)

print(f"Our FFT:\n{a}\n")
print(f"NumPy FFT:\n{b}\n")
print(f"Are they the same?\n{np.allclose(a, b)}\n")